<a href="https://colab.research.google.com/github/yumnaehab-tech/data-science-projects/blob/main/Supermarket_Sales_SQL_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supermarket Sales SQL Analysis

This project demonstrates SQL skills on a supermarket sales dataset.  
All analyses are performed using **SQL Queries** executed in Python via SQLite.  

We will answer business questions like:
- Which product lines generate the most revenue?
- Which cities have the highest sales?
- Which customers purchased multiple times?
- Which branches have the highest average rating?


In [1]:
import zipfile

with zipfile.ZipFile("/content/Super market sales .zip", "r") as zip_ref:
    zip_ref.extractall("/content/")


In [2]:
!ls /content


 sample_data  'SuperMarket Analysis.csv'  'Super market sales .zip'


In [3]:
import pandas as pd
import sqlite3


In [4]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv("/content/SuperMarket Analysis.csv")

# Display first few rows
df.head()


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3


In [5]:
# Connect to SQLite database (it will be created if it doesn’t exist)
conn = sqlite3.connect("supermarket_sales.db")

# Save the DataFrame to a new SQL table called 'sales'
df.to_sql("sales", conn, if_exists="replace", index=False)

# Confirm table creation by listing tables in the database
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables


,name
0,sales


In [6]:
# Preview 5 rows directly from SQL
pd.read_sql("SELECT * FROM sales LIMIT 5;", conn)


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Sales,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,Alex,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,1:08:00 PM,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,Giza,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29:00 AM,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,Alex,Yangon,Normal,Female,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,1:23:00 PM,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,Alex,Yangon,Member,Female,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,8:33:00 PM,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,Alex,Yangon,Member,Female,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37:00 AM,Ewallet,604.17,4.761905,30.2085,5.3


In [7]:
query1 = """
SELECT [Product line] AS Product, ROUND(SUM(Sales), 2) AS Total_Sales
FROM sales
GROUP BY [Product line]
ORDER BY Total_Sales DESC;
"""
pd.read_sql(query1, conn)


,Product,Total_Sales
0,Food and beverages,56144.84
1,Sports and travel,55122.83
2,Electronic accessories,54337.53
3,Fashion accessories,54305.90
4,Home and lifestyle,53861.91
5,Health and beauty,49193.74


In [8]:
query2 = """
SELECT City, ROUND(SUM(Sales), 2) AS Total_Sales
FROM sales
GROUP BY City
ORDER BY Total_Sales DESC;
"""
pd.read_sql(query2, conn)


,City,Total_Sales
0,Naypyitaw,110568.71
1,Yangon,106200.37
2,Mandalay,106197.67


In [9]:
query3 = """
SELECT [Product line], ROUND(AVG(Rating), 2) AS Avg_Rating
FROM sales
GROUP BY [Product line]
ORDER BY Avg_Rating DESC;
"""
pd.read_sql(query3, conn)


,Product line,Avg_Rating
0,Food and beverages,7.11
1,Fashion accessories,7.03
2,Health and beauty,7.00
3,Sports and travel,6.92
4,Electronic accessories,6.92
5,Home and lifestyle,6.84


In [10]:
query4 = """
SELECT Payment, COUNT(*) AS Count
FROM sales
GROUP BY Payment
ORDER BY Count DESC;
"""
pd.read_sql(query4, conn)


,Payment,Count
0,Ewallet,345
1,Cash,344
2,Credit card,311


In [11]:
query5 = """
SELECT Gender, ROUND(SUM(Sales), 2) AS Total_Sales
FROM sales
GROUP BY Gender
ORDER BY Total_Sales DESC;
"""
pd.read_sql(query5, conn)


,Gender,Total_Sales
0,Female,194671.84
1,Male,128294.91


In [12]:
conn.close()
